In [26]:
!pip install spotipy
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from urllib.request import Request, urlopen
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [53]:
song = []
rank = []
year = []
artist = []


for i in range(2019,2020):
    print(i)
    
#     page = Request("https://www.billboard.com/charts/year-end/2019/hot-100-songs",headers={'User-Agent': 'Mozilla/5.0'})
    page = Request("https://www.billboard.com/charts/year-end/" + str(i) + "/hot-100-songs",headers={'User-Agent': 'Mozilla/5.0'})

    html = urlopen(page).read()
    page_soup = soup(html,"html.parser")

    #splits the page into the 100 artists
    containers = page_soup.main.findAll("div",{"class":"ye-chart-item__primary-row"})
    
    #Sleep because if not denies too many requests
    time.sleep(2)


    #iterates through the 100 artists
    for container in containers:

        rank.append(container.contents[1].text.strip())
        song.append(container.find("div",{"class":"ye-chart-item__title"}).text.strip())
        artist.append(container.find("div",{"class":"ye-chart-item__artist"}).text.strip())
        year.append(int(i))
        
data = pd.DataFrame()

data['song'] = song
data['artist'] = artist
data['rank'] = rank
data['year'] = year


data.astype({'year': 'int32'}).dtypes
data.astype({'rank': 'int32'}).dtypes
data.info()
    
    





2019
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
song      100 non-null object
artist    100 non-null object
rank      100 non-null object
year      100 non-null int64
dtypes: int64(1), object(3)
memory usage: 3.2+ KB


In [54]:
SPOTIPY_CLIENT_ID='dfc90bae743943689c070c2f0b7fd7e0'
SPOTIPY_CLIENT_SECRET='477f42668a624f6e875a5399643b8c00'

# passes in the credentials for the cleint credential flow
client_credentials_manager = SpotifyClientCredentials(SPOTIPY_CLIENT_ID,SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [55]:
data.head()

,song,artist,rank,year
0,Old Town Road,Lil Nas X Featuring Billy Ray Cyrus,1,2019
1,Sunflower (Spider-Man: Into The Spider-Verse),Post Malone & Swae Lee,2,2019
2,Without Me,Halsey,3,2019
3,Bad Guy,Billie Eilish,4,2019
4,Wow.,Post Malone,5,2019


In [79]:
song_ids = []
artist_ids = []
for index,row in data.iterrows():
    q = ' '.join([row.song, row.artist])
    result = sp.search(q, type='track')['tracks']['items']
    if result == []:
        result = sp.search(row.song, type='track')['tracks']['items']
    result = result[0]
    artist_ids.append([artist['id'] for artist in result['artists']])
    song_ids.append(result['id'])

In [80]:
data.insert(1, 'song_id', song_ids)
data.insert(3, 'artist_ids', artist_ids)

In [81]:
data.head()

,song,song_id,artist,artist_ids,rank,year
0,Old Town Road,2YpeDb67231RjR0MgVLzsG,Lil Nas X Featuring Billy Ray Cyrus,"[7jVv8c5Fj3E9VhNjxT4snq, 60rpJ9SgigSd16DOAG7GSa]",1,2019
1,Sunflower (Spider-Man: Into The Spider-Verse),3KkXRkHbMCARz0aVfEt68P,Post Malone & Swae Lee,"[246dkjvS1zLTtiykXe5h60, 1zNqQNIdeOUZHb8zbZRFMX]",2,2019
2,Without Me,5p7ujcrUXASCNwRaWNHR1C,Halsey,[26VFTg2z8YR0cCuwLzESi2],3,2019
3,Bad Guy,2Fxmhks0bxGSBdJ92vM42m,Billie Eilish,[6qqNVTkY8uBg9cP3Jd7DAH],4,2019
4,Wow.,7xQAfvXzm3AkraOtGPWIZg,Post Malone,[246dkjvS1zLTtiykXe5h60],5,2019
